In [2]:
from sqlalchemy import create_engine
from key import DB_USER, DB_PASSWORD, DB_ENDPOINT

In [93]:
#assign import secret values to varibles
user = DB_USER
pwd = DB_PASSWORD
endpoint = DB_ENDPOINT

#create url for engin
dialect = f"mysql+pymysql://{user}:{pwd}@{endpoint}/boulderinggyms"

#create engin
engine = create_engine(dialect, echo = True, future=True)

In [117]:
from sqlalchemy import Table, Column, ForeignKey, text
from sqlalchemy.dialects.mysql import BIGINT, VARCHAR, DECIMAL, TIMESTAMP
from sqlalchemy.orm import declarative_base, relationship, Session

Base = declarative_base()
metadata = Base.metadata

In [118]:
gyms_table = Table(
    "gyms",
    metadata,
    Column('gymID', BIGINT(unsigned=True), primary_key=True, autoincrement=True),
    Column('gymName', VARCHAR(80), nullable=False),
    Column('gymNameFromGoogle', VARCHAR(80)),
    Column('gymAddress', VARCHAR(100)),
    Column('locLatitude', DECIMAL(8,6)),
    Column('locLongitude', DECIMAL(8,6)),
    Column('ratingFromMP', DECIMAL(8,6)),
    Column('googlePlaceID', VARCHAR(255)),
    Column('lastUpdated', TIMESTAMP, nullable=False, server_default=text("CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"))
)
gyms_table

Table('gyms', MetaData(), Column('gymID', BIGINT(unsigned=True), table=<gyms>, primary_key=True, nullable=False), Column('gymName', VARCHAR(length=80), table=<gyms>, nullable=False), Column('gymNameFromGoogle', VARCHAR(length=80), table=<gyms>), Column('gymAddress', VARCHAR(length=100), table=<gyms>), Column('locLatitude', DECIMAL(precision=8, scale=6), table=<gyms>), Column('locLongitude', DECIMAL(precision=8, scale=6), table=<gyms>), Column('ratingFromMP', DECIMAL(precision=8, scale=6), table=<gyms>), Column('googlePlaceID', VARCHAR(length=255), table=<gyms>), Column('lastUpdated', TIMESTAMP(), table=<gyms>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000019AE9E25B20>, for_update=False)), schema=None)

In [119]:
photos_table = Table(
    "photos",
    metadata,
    Column('photoID', BIGINT(unsigned=True), primary_key=True, autoincrement=True),
    Column('gymID', BIGINT(unsigned=True), ForeignKey('gyms.gymID'), nullable=False),
    Column('photoGoogleReference', VARCHAR(255)),
    Column('photoURL', VARCHAR(160)),
    Column('lastUpdated', TIMESTAMP, nullable=False, server_default=text("CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"))
)

photo_table

Table('photos', MetaData(), Column('photoID', BIGINT(unsigned=True), table=<photos>, primary_key=True, nullable=False), Column('gymID', BIGINT(unsigned=True), ForeignKey('gyms.gymID'), table=<photos>, nullable=False), Column('photoGoogleReference', VARCHAR(length=255), table=<photos>), Column('photoURL', VARCHAR(length=160), table=<photos>), Column('lastUpdated', TIMESTAMP(), table=<photos>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000019AE9D9F880>, for_update=False)), schema=None)

In [115]:
metadata.create_all(engine)

2021-05-11 10:00:49,867 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-05-11 10:00:49,869 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-11 10:00:49,906 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-05-11 10:00:49,908 INFO sqlalchemy.engine.Engine [generated in 0.00173s] {}
2021-05-11 10:00:49,972 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-05-11 10:00:49,973 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-11 10:00:50,067 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-11 10:00:50,071 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-05-11 10:00:50,072 INFO sqlalchemy.engine.Engine [generated in 0.00183s] {'table_schema': 'boulderinggyms', 'table_name': 'gyms'}
2021-05-11 10:00:50,109 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_nam

In [120]:
class Gym(Base):
    __table__ = gyms_table
    
    photo = relationship("Photo", back_populates="gym")
    
    def __repr__(self):
        return f"Gym({self.gymID!r}, {self.gymName!r}, {self.gymNameFromGoogle!r}, {self.gymAddress!r},  {self.locLatitude!r}, {self.locLongitude}, {self.ratingFromMP!r}, {self.googlePlaceID!r}, {self.lastUpdated!r})"
    
class Photo(Base):
    __table__ = photos_table
    
    gym = relationship("Gym", back_populates="photo")
    
    def __repr__(self):
        return f"Photo({self.photoID!r}, {self.gymID!r}, {self.photoGoogleReference!r}, {self.photoURL!r}, {self.lastUpdated!r})"

In [121]:
fakeGym = Gym(gymName = "phoneyName", gymAddress = "Nowhere")
fakePhoto = Photo(photoURL = "fakeurl.fake/photo", gym = fakeGym)

In [124]:
fakePhoto.gym

Gym(None, 'phoneyName', None, 'Nowhere',  None, None, None, None, None)

In [123]:
session = Session(engine)

In [125]:
session.add(fakeGym)

In [127]:
fakePhoto in session

True

In [128]:
session.flush()

2021-05-11 10:24:32,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-11 10:24:32,625 INFO sqlalchemy.engine.Engine INSERT INTO gyms (`gymName`, `gymNameFromGoogle`, `gymAddress`, `locLatitude`, `locLongitude`, `ratingFromMP`, `googlePlaceID`) VALUES (%(gymName)s, %(gymNameFromGoogle)s, %(gymAddress)s, %(locLatitude)s, %(locLongitude)s, %(ratingFromMP)s, %(googlePlaceID)s)
2021-05-11 10:24:32,627 INFO sqlalchemy.engine.Engine [generated in 0.00216s] {'gymName': 'phoneyName', 'gymNameFromGoogle': None, 'gymAddress': 'Nowhere', 'locLatitude': None, 'locLongitude': None, 'ratingFromMP': None, 'googlePlaceID': None}
2021-05-11 10:24:32,666 INFO sqlalchemy.engine.Engine INSERT INTO photos (`gymID`, `photoGoogleReference`, `photoURL`) VALUES (%(gymID)s, %(photoGoogleReference)s, %(photoURL)s)
2021-05-11 10:24:32,668 INFO sqlalchemy.engine.Engine [generated in 0.00174s] {'gymID': 1, 'photoGoogleReference': None, 'photoURL': 'fakeurl.fake/photo'}


In [131]:
session.commit()

2021-05-11 10:24:56,314 INFO sqlalchemy.engine.Engine COMMIT
